In [ ]:
# --------------------------------------------------------------------------------------
# COLAB-SPECIFICS
# --------------------------------------------------------------------------------------

In [ ]:
IN_COLAB = True
REMOUNT = False

In [ ]:
# RUN IN COLAB ONLY: mount Google drive
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# RUN IN COLAB ONLY: unmount, remount Google drive - if required
if REMOUNT:
    !fusermount -u /content/drive 2>/dev/null || true
    !rm -rf /content/drive
    from google.colab import auth
    auth.authenticate_user()
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

In [ ]:
# RUN IN COLAB ONLY: install required modules: ssm, openpyxl
if IN_COLAB:
    !pip -q install git+https://github.com/lindermanlab/ssm.git #egg=ssm
    !pip -q install openpyxl
    !pip -q install filelock pyarrow

In [ ]:
# RUN IN COLAB ONLY: force-fetch latest files from GitHub (overwrite local copies)
if IN_COLAB:
    !rm -rf /content/SLDS
    !git clone --depth=1 https://github.com/chrismader1/SLDS.git /content/SLDS
    import sys
    sys.path.append("/content/SLDS")

In [ ]:
# RUN IN COLAB ONLY: prevent timeouts
if IN_COLAB:
    from IPython.display import Javascript, display
    display(Javascript("""
    (function keepAlive(){
      function clickConnect(){
        // Try shadow-root button (newer Colab)
        const el = document.querySelector('colab-connect-button');
        if (el && el.shadowRoot){
          const btn = el.shadowRoot.querySelector('#connect');
          if (btn){ btn.click(); console.log('keepAlive: clicked shadow connect'); return; }
        }
        // Fallbacks
        const btn2 = document.querySelector('#connect, button#connect');
        if (btn2){ btn2.click(); console.log('keepAlive: clicked #connect'); return; }
        console.log('keepAlive: connect button not found');
      }
      setInterval(clickConnect, 60 * 1000);
      console.log('keepAlive: armed');
    })();
    """))

In [ ]:
# RUN IN COLAB ONLY: clear cache
if IN_COLAB:
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_results.csv"
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_segments.csv"
    !echo "Reset done."

In [ ]:
# --------------------------------------------------------------------------------------
# IMPORT MODULES
# --------------------------------------------------------------------------------------

# BLAS/threads
import os, multiprocessing
n_cpu = multiprocessing.cpu_count()

# Max speed on Mac: let Accelerate (vecLib) use all cores
for k in ["OMP_NUM_THREADS","OPENBLAS_NUM_THREADS","MKL_NUM_THREADS",
          "VECLIB_MAXIMUM_THREADS","NUMEXPR_NUM_THREADS"]:
    os.environ[k] = str(n_cpu)

# TQDM widget off
os.environ["TQDM_NOTEBOOK"] = "0"

# Light / general imports
import numpy as np
import pandas as pd
import tqdm
import tqdm.auto as tqa
from tqdm.std import tqdm as tqdm_std

def _trange_no_widget(n, *a, **k):
    k.setdefault("disable", True)
    return tqdm_std(range(n), *a, **k)
tqdm.tqdm = tqdm_std
tqdm.trange = _trange_no_widget
tqa.tqdm = tqdm_std
tqa.trange = _trange_no_widget

# Import your pipeline AFTER env + tqdm config
from gridsearch import pipeline_actual

In [ ]:
# --------------------------------------------------------------------------------------
# PATHS
# --------------------------------------------------------------------------------------

COLAB_PATHS = {
    "data_excel": "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/bbg_data.xlsx",  # Google Drive
    "ff_dir": "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/",                   # Google Drive
    "ff_files": {
        "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
        "ff3": "F-F_Research_Data_Factors_daily.csv",
        "mom": "F-F_Momentum_Factor_daily.csv",},
    "results_csv": "/content/drive/MyDrive/Colab Notebooks/SLDS/Out/gridsearch_results.csv",            # Google Drive
    "segments_parquet": "/content/drive/MyDrive/Colab Notebooks/SLDS/Out/gridsearch_segments.parquet",  # Google Drive
    "tmp_dir":          "/content/tmp_slds/",                      # Colab local
    "segments_tmp_csv": "/content/tmp_slds/segments_tmp.csv",      # Colab local
}

LOCAL_PATHS = {
    "data_excel": "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx",
    "ff_dir": "/Users/chrismader/Python/SLDS/Data/",
    "ff_files": {
        "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
        "ff3": "F-F_Research_Data_Factors_daily.csv",
        "mom": "F-F_Momentum_Factor_daily.csv",},
    "results_csv": "/Users/chrismader/Python/SLDS/Out/gridsearch_results.csv",
    "segments_parquet": "/Users/chrismader/Python/SLDS/Out/gridsearch_segments.parquet",
    "tmp_dir":          "/Users/chrismader/Python/SLDS/tmp_slds/",
    "segments_tmp_csv": "/Users/chrismader/Python/SLDS/tmp_slds/segments_tmp.csv",
}


In [ ]:
# --------------------------------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------------------------------

CONFIG = {
    
    # Core defaults
    # "n_jobs": -1,  # multi-threading
    "dt": 1.0 / 252.0,
    "n_iters": 50,
    "h_z": 3.0,  # CUSUM parameter
    
    # Batch windows
    "batch_grid": [
        {"train_window": 1260, "overlap_window": 1008},
    ],

    # Number of regimes
    "K_grid": [2, 5, 6],
    
    # Unrestricted models: 
    "unrestricted_models": [
        {"label": "[y]",         "channels": ["y"],                "dim_latent": [1]},
        {"label": "[y,h]",       "channels": ["y","h"],            "dim_latent": [2]},
        {"label": "[g,v]",       "channels": ["g","v"],            "dim_latent": [2]},
        {"label": "[g,v,h]",     "channels": ["g","v","h"],        "dim_latent": [2,3]},
        {"label": "[y,g,v,h]",   "channels": ["y","g","v","h"],    "dim_latent": [3,4]},
    ],

    # Restricted models: 
    "restricted_models": [
        {"label": "fund1",        "channels": ["y"],                 "dim_latent": [2],    "C_type": "fund1"},
        {"label": "fund1_vix",    "channels": ["y","h"],             "dim_latent": [3],    "C_type": "fund1_vix"},
        {"label": "fund2",        "channels": ["y","g"],             "dim_latent": [2],    "C_type": "fund2"},
        {"label": "fund2_vix",    "channels": ["y","g","h"],         "dim_latent": [3],    "C_type": "fund2_vix"},
        {"label": "fund3",        "channels": ["y","v","g"],         "dim_latent": [2],    "C_type": "fund3"},
        {"label": "fund3_vix",    "channels": ["y","v","g","h"],     "dim_latent": [3],    "C_type": "fund3_vix"},   

        {"label": "factor1",      "channels": ["y"],                 "dim_latent": [2],    "C_type": "factor1"},
        {"label": "factor1_vix",  "channels": ["y","h"],             "dim_latent": [3],    "C_type": "factor1_vix"},

        {"label": "factor2_ff3",   "channels": ["y","mkt","smb","hml"],                   "dim_latent": [3], "C_type": "factor2"},
        {"label": "factor2_ff3mom","channels": ["y","mkt","smb","hml","mom"],             "dim_latent": [4], "C_type": "factor2"},
        {"label": "factor2_ff5",   "channels": ["y","mkt","smb","hml","rmw","cma"],       "dim_latent": [5], "C_type": "factor2"},
        {"label": "factor2_ff5mom","channels": ["y","mkt","smb","hml","rmw","cma","mom"], "dim_latent": [6], "C_type": "factor2"},
    ],

    # Model selection
    "run_unrestricted": True,
    "run_restricted": True,
    "predict_oos": True,

    # Output
    "verbose": False,
    "display": False,
}

PATHS = COLAB_PATHS if IN_COLAB else LOCAL_PATHS
for k, v in PATHS.items(): 
    CONFIG[k] = v
# per-security temp file templates used by IOManager
CONFIG["tmp_results_fmt"]  = "{tmp_dir}/tmp_res_{security}.csv"
CONFIG["tmp_segments_fmt"] = "{tmp_dir}/tmp_seg_{security}.csv"


In [ ]:
# --------------------------------------------------------------------------------------
# EXECUTE
# --------------------------------------------------------------------------------------

def main():
    
    import os
    import pandas as pd

    # IO from CONFIG
    csv_path = CONFIG["results_csv"]
    filename = CONFIG["data_excel"]
    out_path = os.path.dirname(CONFIG["results_csv"]) + "/"
    
    securities_master = [
        "MSFT","NVDA","AAPL","AVGO","GOOGL","ORCL","IBM","CSCO","CRM","AMD","INTU","NOW","TXN","QCOM",
        "ADBE","AMAT","PLTR","ACN","META","NFLX","GOOGL","DIS","VZ","T","AMZN","TSLA","HD",
        "MCD","BKNG","COST","WMT","PG","KO","PM","PEP","JPM","V","MA","BAC","WFC","GS","MS","SPGI",
        "SCHW","AXP","BLK","C","PGR","LLY","JNJ","ABBV","UNH","ABT","MRK","ISRG","BSX","TMO","AMGN","RTX",
        "XOM","CVX","GE","CAT","BA","HON","LIN","NEE","DUK","SO","AEP","D","AMT","PLD","EQIX","O","CBRE",]
    
    # resume if CSV exists; otherwise start from beginning
    if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
        df = pd.read_csv(csv_path)
        assert "security" in df.columns, "results_csv missing 'security' column"
        done_set = set(df["security"].dropna().astype(str))
        done = [s for s in securities_master if s in done_set]
        securities = [s for s in securities_master if s not in done_set]
        print("Resuming from CSV:", csv_path)
        print("Already completed:", done)
    else:
        securities = securities_master
        print("Resuming from: START (no/empty CSV)")
    
    print("Remaining tickers:", len(securities), securities)

    if len(securities) == 0:
        print("Nothing to do.")
        return

    _ = pipeline_actual(securities=securities, CONFIG=CONFIG)

if __name__ == "__main__":
    main()
